Children of a node at index i:

   - Left child: 2 * i + 1
   -  Right child: 2 * i + 2

Grandchildren of a node at index i:

    -  A child at 2 * i + 1 will have children at:
        - 2 * (2 * i + 1) + 1 = 4 * i + 3
        - 2 * (2 * i + 1) + 2 = 4 * i + 4
    - A child at 2 * i + 2 will have children at:
        - 2 * (2 * i + 2) + 1 = 4 * i + 5
        - 2 * (2 * i + 2) + 2 = 4 * i + 6

(index-1/2)+(index-1/2)

In [ ]:
import math

class MinMaxPriorityHeap(object):
    def __init__(self):
        self.heap = []

    def _level(self, index):
        return int(math.log2(index + 1))

    def _size(self):
        return len(self.heap)

    def _left_child(self, index):
        return 2 * index + 1

    def _right_child(self, index):
        return 2 * index + 2

    def _parent(self, index):
        return (index - 1) // 2

    def peekmin(self):
        assert self._size() > 0
        return self.heap[0]

    def peekmax(self):
        size = self._size()
        assert size > 0
        if size == 1:
            return self.heap[0]
        elif size == 2:
            return self.heap[1]
        else:
            return max(self.heap[1], self.heap[2])

    def push(self, value):
        self.heap.append(value)
        self.heapify_up(self._size() - 1)

    def heapify_up(self, i):
        parent_index = self._parent(i)
        if self._level(i) % 2 == 0:  # Min level
            if i > 0 and self.heap[i] > self.heap[parent_index]:
                self.heap[i], self.heap[parent_index] = self.heap[parent_index], self.heap[i]
                self._heapify_up_max(parent_index)
            else:
                self._heapify_up_min(i)
        else:  # Max level
            if i > 0 and self.heap[i] < self.heap[parent_index]:
                self.heap[i], self.heap[parent_index] = self.heap[parent_index], self.heap[i]
                self._heapify_up_min(parent_index)
            else:
                self._heapify_up_max(i)
#index-1/2+index-1/2

    def _heapify_up_min(self, i):
        #parent index=index-1//2
        #parent of parent i.e grandparent index=(index-1//2)-1//2
        #index-3/4
        while i > 2 and self.heap[i] < self.heap[(i - 3) // 4]:
            self.heap[i], self.heap[(i - 3) // 4] = self.heap[(i - 3) // 4], self.heap[i]
            i = (i - 3) // 4

    def _heapify_up_max(self, i):
        while i > 2 and self.heap[i] > self.heap[(i - 3) // 4]:
            self.heap[i], self.heap[(i - 3) // 4] = self.heap[(i - 3) // 4], self.heap[i]
            i = (i - 3) // 4

    def heapify_down(self, index):
        if self._level(index) % 2 == 0:
            self._heapify_down_min(index)
        else:
            self._heapify_down_max(index)

    def _heapify_down_min(self, index):
        left = self._left_child(index)
        size = self._size()
        if size > left:  # If the node has children
            right = self._right_child(index)
            smallest = left  # Assume left child is smallest

            # Find the smaller of left and right child
            if right < size and self.heap[right] < self.heap[smallest]:
                smallest = right

            child = True  # Assume the smallest node is a direct child
            for j in range(index * 4 + 3, min(index * 4 + 7, size)):  
                if self.heap[j] < self.heap[smallest]:  # Checking grandchildren
                    smallest = j
                    child = False  # Smallest node is a grandchild

            if child:  
                # Swap with the smallest child if it's smaller than the parent
                if self.heap[smallest] < self.heap[index]:
                    self.heap[smallest], self.heap[index] = self.heap[index], self.heap[smallest]
            else:  
                # Swap with the smallest grandchild if needed
                if self.heap[smallest] < self.heap[index]:
                    self.heap[smallest], self.heap[index] = self.heap[index], self.heap[smallest]

                # Ensure the grandchild is still smaller than its parent
                if self.heap[smallest] > self.heap[(smallest - 1) // 2]:
                    self.heap[smallest], self.heap[(smallest - 1) // 2] = self.heap[(smallest - 1) // 2], self.heap[smallest]

                # Recursively heapify down the grandchild
                self._heapify_down_min(smallest)


    def _heapify_down_max(self, index):
        size = self._size()
        left = self._left_child(index)
        if size > left:
            right = self._right_child(index)
            largest = left
            if right < size and self.heap[right] > self.heap[largest]:
                largest = right
            child = True
            for j in range(index * 4 + 3, min(index * 4 + 7, size)):
                if self.heap[j] > self.heap[largest]:
                    largest = j
                    child = False
            if self.heap[largest] > self.heap[index]:
                self.heap[largest], self.heap[index] = self.heap[index], self.heap[largest]
                if not child and self.heap[largest] < self.heap[(largest - 1) // 2]:
                    self.heap[largest], self.heap[(largest - 1) // 2] = self.heap[(largest - 1) // 2], self.heap[largest]
                self._heapify_down_max(largest)

    def pop_min(self):
        assert self._size() > 0
        value = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        if self._size() > 0:
            self.heapify_down(0)
        return value

    def pop_max(self):
        """
        Removes and returns the largest element from the heap.
        Complexity: O(log(n))
        """
        size = self._size()
        
        if size == 1:
            return self.heap.pop()  # If only one element exists, remove and return it
        
        elif size == 2:
            return self.heap.pop()  # If two elements exist, the second one is the max
        
        else:
            # Find the maximum of the two possible max candidates
            i = 1 if self.heap[1] > self.heap[2] else 2
            
            elem = self.heap[i]  # Store the max element to return later
            
            self.heap[i] = self.heap[size - 1]  # Replace the max element with the last element in the heap
            self.heap.pop()  # Remove the last element (which was moved)
            
            self.heapify_down(i)  # Restore the heap property starting from the replaced position
            
            return elem  # Return the removed max element


# Example Usage
mhp = MinMaxPriorityHeap()
mhp.push(10)
mhp.push(20)
mhp.push(30)
mhp.push(40)
mhp.push(50)
print(mhp.heap)


[10, 50, 30, 20, 40]
